<a href="https://colab.research.google.com/github/JaiderSantos/metodos1_JaiderSantos_NicolasGomez/blob/main/tarea4/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte 2 Tarea 4 Algebra lineal



> Punto 8



In [9]:
A = np.array([[4, -2, 1],
              [20, -7, 12],
              [-8, 13, 17]], dtype=float)

# Inicializamos L y U
n = A.shape[0]
L = np.eye(n)
U = A.copy()

# Realizamos la descomposición LU
for i in range(n):
    for j in range(i+1, n):
        # Calculamos el multiplicador
        l_ij = U[j, i] / U[i, i]
        L[j, i] = l_ij

        # Actualizamos la fila j de U
        U[j, :] = U[j, :] - l_ij * U[i, :]

# Mostramos los resultados
print("Matriz L:")
print(L)
print("\nMatriz U:")
print(U)

A_reconstructed = np.dot(L, U)
print("\nMatriz A reconstruida:")
print(A_reconstructed)

Matriz L:
[[ 1.  0.  0.]
 [ 5.  1.  0.]
 [-2.  3.  1.]]

Matriz U:
[[ 4. -2.  1.]
 [ 0.  3.  7.]
 [ 0.  0. -2.]]

Matriz A reconstruida:
[[ 4. -2.  1.]
 [20. -7. 12.]
 [-8. 13. 17.]]


Punto 10

In [2]:
import numpy as np

def jacobi_method(A, max_iterations=100, tol=1e-10):
    n = A.shape[0]
    # Inicializar los vectores propios y la matriz diagonal
    V = np.eye(n)  # Matriz de vectores propios
    D = np.copy(A)  # Matriz diagonalizada

    for _ in range(max_iterations):
        # Encontrar el elemento más grande fuera de la diagonal
        off_diag = np.abs(D - np.diag(np.diag(D)))
        i, j = np.unravel_index(np.argmax(off_diag), off_diag.shape)

        if off_diag[i, j] < tol:
            break

        # Calcular el ángulo theta
        if D[i, i] == D[j, j]:
            theta = np.pi / 4  # 45 grados si son iguales
        else:
            theta = 0.5 * np.arctan2(2 * D[i, j], D[j, j] - D[i, i])

        # Crear la matriz de rotación
        c = np.cos(theta)
        s = np.sin(theta)

        # Actualizar la matriz D y la matriz de vectores propios V
        R = np.eye(n)
        R[i, i] = c
        R[j, j] = c
        R[i, j] = -s
        R[j, i] = s

        D = R.T @ D @ R
        V = V @ R

    return np.diagonal(D), V  # Retornar valores propios y vectores propios

# Definimos la matriz A
A = np.array([[4, 1, 1],
              [1, 3, 2],
              [1, 2, 5]], dtype=float)

# Usamos el método de Jacobi
eigenvalues_jacobi, eigenvectors_jacobi = jacobi_method(A)

print("Valores propios (Jacobi):", eigenvalues_jacobi)
print("Vectores propios (Jacobi):")
print(eigenvectors_jacobi)

eigenvalues_np, eigenvectors_np = np.linalg.eig(A)

print("\nValores propios (NumPy):", eigenvalues_np)
print("Vectores propios (NumPy):")
print(eigenvectors_np)

Valores propios (Jacobi): [3.3962362 6.2966035 2.3071603]
Vectores propios (Jacobi):
[[-0.89537629  0.34752195 -0.27844171]
 [ 0.09607536  0.76130557  0.6412358 ]
 [ 0.43482275  0.54739595 -0.71504325]]

Valores propios (NumPy): [6.89510652 3.39729507 1.70759841]
Vectores propios (NumPy):
[[ 0.43170413  0.88573564  0.17059871]
 [ 0.49725362 -0.07589338 -0.86427949]
 [ 0.75257583 -0.45794385  0.47319874]]


Punto 13

In [8]:
import numpy as np

def jacobiano_central(func, x, h=1e-5):
    n = len(x)
    m = len(func(x))
    J = np.zeros((m, n))

    for i in range(n):
        # Crear un vector perturbado
        x_plus = np.array(x, dtype=float)
        x_minus = np.array(x, dtype=float)

        x_plus[i] += h
        x_minus[i] -= h

        for j in range(m):
            # The problem was in this line. func returns an array,
            #  and you need to select the j-th element of that array
            #  using func(arguments)[j]  for each term in the expression.
            J[j, i] = (-func(x_plus)[j] + 8 * func(x_plus + h * np.eye(n)[i])[j] \
                       - 8 * func(x_minus - h * np.eye(n)[i])[j] + func(x_minus)[j]) / (12 * h)

    return J


def f(x):
    return np.array([x[0]**2 + x[1], x[1]**2 + x[2], x[0] + x[2]**2])

x_point = np.array([0.5, 0.5, 0.5])
h = 0.01
jacobian_estimation = jacobiano_central(f, x_point, h)
print(jacobian_estimation)



def jacobiano_central_Oh2(func, x, h=1e-5):
    n = len(x)
    m = len(func(x))
    J = np.zeros((m, n))

    for i in range(n):
        # Crear un vector perturbado
        x_plus = np.array(x, dtype=float)
        x_minus = np.array(x, dtype=float)

        x_plus[i] += h
        x_minus[i] -= h

        for j in range(m):
            J[j, i] = (func(x_plus)[j] - func(x_minus)[j]) / (2 * h)

    return J
jacobian_Oh2_estimation = jacobiano_central_Oh2(f, x_point, h)
print(jacobian_Oh2_estimation)


h_values = [0.01, 0.001, 0.0001]
for h in h_values:
    J_Oh4 = jacobiano_central(f, x_point, h)
    J_Oh2 = jacobiano_central_Oh2(f, x_point, h)
    error = np.linalg.norm(J_Oh4 - J_Oh2)
    print(f"h = {h}, error = {error}")

[[2.5 2.5 0. ]
 [0.  2.5 2.5]
 [2.5 0.  2.5]]
[[1. 1. 0.]
 [0. 1. 1.]
 [1. 0. 1.]]
h = 0.01, error = 3.6742346141747775
h = 0.001, error = 3.674234614174781
h = 0.0001, error = 3.6742346141748152
